# Vertex Forecast - experiments with scenarios

In [1]:
import os

GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
REGION = 'us-central1'
BQ_LOCATION='US'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"REGION: {REGION}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
REGION: us-central1


In [2]:
import google.cloud.aiplatform as vertex_ai
from google.cloud import bigquery
from google.cloud import storage

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

In [3]:
bq_client = bigquery.Client(
    project=PROJECT_ID, 
    location=BQ_LOCATION
)

storage_client = storage.Client(project=PROJECT_ID)

vertex_ai.init(
    project=PROJECT_ID,
    location=REGION
)

In [44]:
# previously defined
BQ_DATASET="m5_us"
BQ_TABLE="sdk_train"
BQ_TABLE_PLAN="sdk_plan"

# new vars
EXPERIMENT="m5_nb3" # TODO - update
VERSION="v2"

## get dataset

In [5]:
dataset = vertex_ai.TimeSeriesDataset('projects/934903580331/locations/us-central1/datasets/462153324456574976')

### define column specs

In [21]:
TARGET_COLUMN = 'gross_quantity'
TIME_COLUMN = 'date'
SERIES_COLUMN = 'timeseries_id'

AVAILABLE_AT_FORECAST_COLS = list(set(dataset.column_names) - set(['splits','timeseries_id','gross_quantity','sell_price']))

UNAVAILABLE_AT_FORECAST_COLS=[
    TARGET_COLUMN,
    'sell_price'
]

COL_TRANSFORMS = {
    TIME_COLUMN:"timestamp",
    TARGET_COLUMN:"numeric",
    "product_id":"categorical",
    "location_id":"categorical",
    "weekday":"categorical",
    "event_name_1":"categorical",
    "year":"categorical",
    "event_type_1":"categorical",
    "month":"categorical",
    "dept_id":"categorical",
    "event_type_2":"categorical",
    "wday":"categorical",
    "state_id":"categorical",
    "snap_WI":"categorical",
    "snap_CA":"categorical",
    "snap_TX":"categorical",
    "event_name_2":"categorical",
    "cat_id":"categorical",
    "sell_price":"numeric"
}

In [26]:
# forecast spec
FORECAST_GRANULARITY = 'DAY'
DATA_GRANULARITY_COUNT=1
FORECAST_HORIZON = 14
# CONTEXT_WINDOW = 14
forecast_test_length = 14
forecast_val_length = 14

# model config
OPTIMIZATION_OBJECTIVE="minimize-rmse"

# job spec
MILLI_NODE_HRS=1000
HOLIDAY_REGIONS=['GLOBAL', 'NA', 'US']

## Train Forecasting Model with AutoML - Multiple Scenarios

The goal here is to train a range of AutoML Forecasting Models on the same data using combination of forecast horizon and context window. This is follow the guidance of [Considerations for setting the context window and forecast horizon](https://cloud.google.com/vertex-ai/docs/datasets/bp-tabular#context-window)

*Note, there may be a default concurrent training job limit for Vertex AI AutoML of 5 jobs* ([reference here](https://cloud.google.com/vertex-ai/docs/quotas#tabular)). 

Here we are running 5 jobs but it we needed to run more we would have to wait for a job to complete or submit a quota increase ([reference here](https://cloud.google.com/vertex-ai/docs/quotas#quota_increases)).

In [30]:
def scenario(ds, cw):
    
    forecast_job = vertex_ai.AutoMLForecastingTrainingJob(
        display_name = f'{EXPERIMENT}_{VERSION}_CW{cw}',
        optimization_objective=OPTIMIZATION_OBJECTIVE,
        column_specs = COL_TRANSFORMS,
        labels = {'experiment':f'{EXPERIMENT}', 'cw':f'{cw}'}
    )
    
    forecast=forecast_job.run(
        dataset=dataset,
        target_column=TARGET_COLUMN,
        time_column=TIME_COLUMN,
        time_series_identifier_column = SERIES_COLUMN,
        unavailable_at_forecast_columns = UNAVAILABLE_AT_FORECAST_COLS,
        available_at_forecast_columns = AVAILABLE_AT_FORECAST_COLS,
        forecast_horizon = FORECAST_HORIZON,
        data_granularity_unit = FORECAST_GRANULARITY.lower(),
        data_granularity_count = DATA_GRANULARITY_COUNT,
        predefined_split_column_name = "splits",
        context_window = cw,
        export_evaluated_data_items = True,
        export_evaluated_data_items_bigquery_destination_uri = f"bq://{PROJECT_ID}:{BQ_DATASET}:{BQ_TABLE}_automl_cw{cw}",
        validation_options = "fail-pipeline",
        budget_milli_node_hours = MILLI_NODE_HRS,
        model_display_name = f"{EXPERIMENT}_{BQ_TABLE}_CW{cw}",
        model_labels = {'experiment':f'{EXPERIMENT}', 'cw':f'{cw}'},
        holiday_regions = ['GLOBAL', 'NA', 'US'],
        sync = False
    )
    
    # FORECAST_MODEL_RSC_NAME = forecast.resource_name
    # print(f"FORECAST_MODEL_RSC_NAME: {FORECAST_MODEL_RSC_NAME}")
    
    # TODO - update - return tuple with CW?
    return forecast

In [31]:
windows = [28, 14, 7]

AutoMLForecastingTrainingJob projects/934903580331/locations/us-central1/trainingPipelines/5891482781103423488 current state:
PipelineState.PIPELINE_STATE_RUNNING


In [32]:
scenarios = []
for w in windows:
    scenarios.append(scenario(dataset, w))

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4315222911523749888?project=934903580331
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8769845892946591744?project=934903580331
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2917418177178632192?project=934903580331
AutoMLForecastingTrainingJob projects/934903580331/locations/us-central1/trainingPipelines/4315222911523749888 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/934903580331/locations/us-central1/trainingPipelines/8769845892946591744 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/934903580331/locations/us-central1/trainingPipelines/2917418177178632192 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/934903580331/locations/us-central1/trainingPipelines/4315222911523749888 curr

In [35]:
scenarios

 resource name: projects/934903580331/locations/us-central1/models/7186603712213680128,
 resource name: projects/934903580331/locations/us-central1/models/5395296960427065344,
 resource name: projects/934903580331/locations/us-central1/models/6509937868201263104]

In [45]:
# TODO - parameterize

models_obj = []

MODEL_28_RESOURCE_NAME = scenarios[0].resource_name
model_28 = vertex_ai.Model(MODEL_28_RESOURCE_NAME)
models_obj.append((model_28, 28))

MODEL_14_RESOURCE_NAME = scenarios[1].resource_name
model_14 = vertex_ai.Model(MODEL_14_RESOURCE_NAME)
models_obj.append((model_14, 14))

MODEL_7_RESOURCE_NAME = scenarios[2].resource_name
model_7 = vertex_ai.Model(MODEL_7_RESOURCE_NAME)
models_obj.append((model_7, 7))

models_obj

[(<google.cloud.aiplatform.models.Model object at 0x7f448bc9c390> 
  resource name: projects/934903580331/locations/us-central1/models/7186603712213680128,
  28),
 (<google.cloud.aiplatform.models.Model object at 0x7f448bc5ad10> 
  resource name: projects/934903580331/locations/us-central1/models/5395296960427065344,
  14),
 (<google.cloud.aiplatform.models.Model object at 0x7f448bc5ac90> 
  resource name: projects/934903580331/locations/us-central1/models/6509937868201263104,
  7)]

## retreive default model evaluation

In [52]:
import time

# # Create and log experiment
vertex_ai.init(experiment=EXPERIMENT.replace("_","-"))

for model, cw in models_obj:
    # create run name
    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
    EXPERIMENT_RUN_NAME = f"run-{TIMESTAMP}"
    
    model_evaluation = list(model.list_model_evaluations())[0]
    metrics_dict = {k: [v] for k, v in dict(model_evaluation.metrics).items()}
    
    # log params
    params = {}
    params["budget_hrs"] = MILLI_NODE_HRS
    params["horizon"] = FORECAST_HORIZON
    params["context_window"] = cw
    params["opt_objective"] = OPTIMIZATION_OBJECTIVE
    
    # log metrics
    metrics = {}
    metrics["MAE"] = metrics_dict['meanAbsoluteError'][0]
    metrics["RMSE"] = metrics_dict['rootMeanSquaredError'][0]
    metrics["MAPE"] = metrics_dict['meanAbsolutePercentageError'][0]
    metrics["rSquared"] = metrics_dict['rSquared'][0]
    metrics["RMSLE"] = metrics_dict['rootMeanSquaredLogError'][0]
    metrics["WAPE"] = metrics_dict['weightedAbsolutePercentageError'][0]
    
    # log to experiment artifact
    with vertex_ai.start_run(EXPERIMENT_RUN_NAME) as my_run:
        my_run.log_metrics(metrics)
        my_run.log_params(params)

        vertex_ai.end_run()
    
    print(EXPERIMENT_RUN_NAME)
    print(params)
    print(metrics)
    time.sleep(3)
    

Associating projects/934903580331/locations/us-central1/metadataStores/default/contexts/m5-nb3-run-20230329132547 to Experiment: m5-nb3
run-20230329132547
{'budget_hrs': 1000, 'horizon': 14, 'context_window': 28, 'opt_objective': 'minimize-rmse'}
{'MAE': 1.0246576, 'RMSE': 2.048384, 'MAPE': 287217660.0, 'rSquared': 0.6981901, 'RMSLE': 0.5037388, 'WAPE': 68.539856}
Associating projects/934903580331/locations/us-central1/metadataStores/default/contexts/m5-nb3-run-20230329132550 to Experiment: m5-nb3
run-20230329132550
{'budget_hrs': 1000, 'horizon': 14, 'context_window': 14, 'opt_objective': 'minimize-rmse'}
{'MAE': 1.0625765, 'RMSE': 2.0592186, 'MAPE': 350589820.0, 'rSquared': 0.69406325, 'RMSLE': 0.519183, 'WAPE': 71.07627}
Associating projects/934903580331/locations/us-central1/metadataStores/default/contexts/m5-nb3-run-20230329132554 to Experiment: m5-nb3
run-20230329132554
{'budget_hrs': 1000, 'horizon': 14, 'context_window': 7, 'opt_objective': 'minimize-rmse'}
{'MAE': 1.0658535, '

### Model feature attributions 

**TODO**

In [39]:
MODEL_28_EVALS = model_28.list_model_evaluations()

for model_evaluation in MODEL_28_EVALS:
    print(model_evaluation.to_dict())

{'name': 'projects/934903580331/locations/us-central1/models/7186603712213680128@1/evaluations/8302332043744039113', 'metricsSchemaUri': 'gs://google-cloud-aiplatform/schema/modelevaluation/forecasting_metrics_1.0.0.yaml', 'metrics': {'meanAbsoluteError': 1.0246576, 'rootMeanSquaredError': 2.048384, 'rSquared': 0.6981901, 'weightedAbsolutePercentageError': 68.539856, 'rootMeanSquaredLogError': 0.5037388, 'rootMeanSquaredPercentageError': 702380800.0, 'meanAbsolutePercentageError': 287217660.0}, 'createTime': '2023-03-29T09:20:20.696303Z', 'modelExplanation': {'meanAttributions': [{'featureAttributions': {'snap_CA': 0.008122412502676448, 'month': 0.005015062030750952, 'dept_id': 0.046573801404751716, 'snap_WI': 0.011550079316175514, 'state_id': 0.011089179830918226, 'location_id': 0.013717990393000848, 'event_type_2': 7.301415637831124e-10, 'product_id': 0.028992821866121433, 'event_type_1': 0.0004927770722828623, 'wday': 0.05963375185924724, 'event_name_1': 0.0025649872933561596, 'year

## Review scenarios

### Review Custom Metrics with SQL

Some common metrics for evaluating forecasting effectiveness are 
- MAPE, or Mean Absolute Percentage Error
    - $\textrm{MAPE} = \frac{1}{n}\sum{\frac{\mid(actual - forecast)\mid}{actual}}$
- MAE, or Mean Absolute Error
     - $\textrm{MAE} = \frac{1}{n}\sum{\mid(actual - forecast)\mid}$
- MAE divided by average demand so it yields a % like MAPE
    - $\textrm{pMAE} = \frac{\sum{\mid(actual - forecast)\mid}}{\sum{actual}}$
- MSE, or Mean Squared Error
    - $\textrm{MSE} = \frac{1}{n}\sum{(actual-forecast)^2}$
- RMSE, or Root Mean Squared Error
    - $\textrm{RMSE} = \sqrt{\frac{1}{n}\sum{(actual-forecast)^2}}$
- RMSE divided by average demand so it yeilds a % like MAPE
    - $\textrm{pRMSE} = \frac{\sqrt{\frac{1}{n}\sum{(actual-forecast)^2}}}{\frac{1}{n}\sum{actual}}$

It can be helpful to explicity caculate these to make comparison between datasets and models fair.  This section demonstration these calculation with SQL.

In [ ]:
# hybrid-vertex.m5_us.sdk_train_automl_cw14

In [57]:
# TODO - parameterize

query = f"""
WITH
    STACKS AS (
        SELECT *, 28 as CW FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}_automl_cw28` UNION ALL
        SELECT *, 14 as CW FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}_automl_cw14` UNION ALL
        SELECT *, 7 as CW FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}_automl_cw7`
    ),
    FORECASTS AS (
        SELECT DATE({TIME_COLUMN}) as {TIME_COLUMN}, 
        DATE(predicted_on_date) as predited_on_date, 
        CAST({TARGET_COLUMN} as INT64) AS {TARGET_COLUMN}, 
        #splits, 
        {SERIES_COLUMN}, 
        CW, 
        predicted_{TARGET_COLUMN}.value as predicted_{TARGET_COLUMN}
        FROM STACKS
    ),
    LEAD_DAYS AS (
        SELECT *, DATE_DIFF({TIME_COLUMN}, predited_on_date, DAY) as prediction_lead_days
        FROM FORECASTS
    ),
    LATEST AS (
        SELECT 
            {SERIES_COLUMN}, 
            {TIME_COLUMN}, 
            min(prediction_lead_days) as prediction_lead_days
        FROM LEAD_DAYS
        GROUP BY {SERIES_COLUMN}, {TIME_COLUMN}
    ),
    DIFFS AS (
        SELECT 
            {SERIES_COLUMN}, 
            CW, 
            {TIME_COLUMN}, 
            'forecast' as time_series_type,
            predicted_{TARGET_COLUMN} as forecast_value,
            {TARGET_COLUMN} as actual_value,
            ({TARGET_COLUMN} - predicted_{TARGET_COLUMN}) as diff
        FROM LATEST
        LEFT OUTER JOIN LEAD_DAYS
        USING ({SERIES_COLUMN}, {TIME_COLUMN}, prediction_lead_days)    
    )
SELECT 
    {SERIES_COLUMN}, 
    CW, 
    time_series_type, 
    AVG(SAFE_DIVIDE(ABS(diff), actual_value)) as MAPE,
    AVG(ABS(diff)) as MAE,
    SAFE_DIVIDE(SUM(ABS(diff)),SUM(actual_value)) as pMAE,
    AVG(POW(diff, 2)) as MSE,
    SQRT(AVG(POW(diff, 2))) as RMSE,
    SAFE_DIVIDE(SQRT( AVG( POW(diff, 2) ) ) , AVG(actual_value) ) as pRMSE
FROM DIFFS
GROUP BY {SERIES_COLUMN}, CW, time_series_type
ORDER BY {SERIES_COLUMN}, CW, time_series_type    
"""
customMetrics = bq_client.query(query = query).to_dataframe()
customMetrics

# print to run in BigQuery console
# print(query)

,timeseries_id,CW,time_series_type,MAPE,MAE,pMAE,MSE,RMSE,pRMSE
0,FOODS_1_001_CA_1,7,forecast,0.472810,0.905466,1.408503,1.014595,1.007271,1.566866
1,FOODS_1_001_CA_1,14,forecast,0.504722,0.965395,1.501726,1.021288,1.010588,1.572026
2,FOODS_1_001_CA_1,28,forecast,0.516518,0.892563,1.388432,0.928325,0.963496,1.498772
3,FOODS_1_001_CA_2,7,forecast,0.490788,0.718280,1.005593,0.656381,0.810173,1.134243
4,FOODS_1_001_CA_2,14,forecast,0.452630,0.689935,0.965910,0.590175,0.768228,1.075520
...,...,...,...,...,...,...,...,...,...
91465,HOUSEHOLD_2_516_WI_2,14,forecast,0.790130,0.290190,2.031330,0.126322,0.355418,2.487925
91466,HOUSEHOLD_2_516_WI_2,28,forecast,0.852261,0.250014,1.750101,0.123070,0.350814,2.455696
91467,HOUSEHOLD_2_516_WI_3,7,forecast,0.819534,0.236555,3.311774,0.082443,0.287129,4.019802
91468,HOUSEHOLD_2_516_WI_3,14,forecast,0.811475,0.251556,3.521783,0.087483,0.295775,4.140856


### Find best context_window scenario with best (minimum) pMAE

In [58]:
bestCW = customMetrics[customMetrics['pMAE'] == customMetrics.groupby([SERIES_COLUMN])['pMAE'].transform('min')].reset_index()
bestCW

,index,timeseries_id,CW,time_series_type,MAPE,MAE,pMAE,MSE,RMSE,pRMSE
0,2,FOODS_1_001_CA_1,28,forecast,0.516518,0.892563,1.388432,0.928325,0.963496,1.498772
1,4,FOODS_1_001_CA_2,14,forecast,0.452630,0.689935,0.965910,0.590175,0.768228,1.075520
2,6,FOODS_1_001_CA_3,7,forecast,0.473867,0.797022,0.858331,0.836548,0.914630,0.984986
3,11,FOODS_1_001_CA_4,28,forecast,0.758530,0.379439,1.770714,0.184100,0.429069,2.002322
4,13,FOODS_1_001_TX_1,14,forecast,0.374977,0.886159,0.729778,1.841098,1.356871,1.117423
...,...,...,...,...,...,...,...,...,...,...
28671,91457,HOUSEHOLD_2_516_TX_2,28,forecast,0.855528,0.264559,1.851912,0.128389,0.358314,2.508201
28672,91460,HOUSEHOLD_2_516_TX_3,28,forecast,0.844306,0.435946,1.220650,0.400848,0.633126,1.772752
28673,91463,HOUSEHOLD_2_516_WI_1,28,forecast,0.846965,0.223121,3.123693,0.080040,0.282913,3.960778
28674,91466,HOUSEHOLD_2_516_WI_2,28,forecast,0.852261,0.250014,1.750101,0.123070,0.350814,2.455696


## Create BigQuery Table with Best Forecast For Each Series based on pMAE


In [60]:
# TODO
tableMap = {
    28 : f'{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}_automl_cw28',
    14 : f'{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}_automl_cw14',
    7 : f'{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}_automl_cw7',
}

In [63]:
# TODO

# query = """"""
# for i, row in bestCW.iterrows():
#     if i> 0: query += f""" UNION ALL\n"""
#     query += f"""SELECT *, {row['CW']} as CW FROM `{tableMap[row['CW']]}` WHERE {SERIES_COLUMN} = '{row[SERIES_COLUMN]}'
#     """
    
# query = f"""CREATE OR REPLACE TABLE `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}_automl_best` AS
# """ + query + f"""ORDER BY {SERIES_COLUMN}, {TIME_COLUMN}
# """
# print(query)

In [64]:
# job = bq_client.query(query = query)
# job.result()
# (job.ended-job.started).total_seconds()

# Using The Results